# NEAK árrés számító Jupyter Notebook alkalmazás v1

# adatforrás és domain-tudás:


Adatforrás: NEAK publikus gyógyszerforgami adatok (http://www.neak.gov.hu/felso_menu/szakmai_oldalak/publikus_forgalmi_adatok/gyogyszer_forgalmi_adatok)<br><br>
ÁFA besoroláshoz: Tájékoztató - VÉGLEGES Áfa kulcsok és adómentes tevékenységek 2022. január 1-jét_l.pdf (forrás NAV) (https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwif8rCupvb5AhXTSfEDHdzRBNgQFnoECCAQAQ&url=https%3A%2F%2Fnav.gov.hu%2Fpfile%2Ffile%3Fpath%3D%2Fugyfeliranytu%2Fadokulcsok_jarulekmertekek%2Fafakulcs_adomen%2FAfa_kulcsok_es_a_tevekenyseg_kozerdeku_vagy_egyeb_sajatos_jellegere_tekintettel_adomentes_tevekenysegek_kore&usg=AOvVaw0YNI9urVaB02qfyJJRLDER)<br><br>
HGYSZ: Támogatott készítmények árképzése a gyógyszertárban (https://hgysz.hu/egyeb/tamogatott-keszitmenyek-arkepzese-a-gyogyszertarban/)

# számítás módja:

- a forrásadatokból a gyári gyógyszerkészítményeket tartalmazó rekordok (venytipus = 'G') kerülnek be a számítási körbe
- de ezt a halmazt is tovább szűrjük, csak azokat vesszük figyelembe, ahol a fogyár és a dobozszám nagyobb, mint nulla;
- például 2022 1-7 hónap esetében a 817455 adatrekordból így 690163 adatrekordot veszünk csak figyelembe (85 %-ot) ... a többi adat vagy magisztrális vagy sztornó
- egy egy visszaszámoló algoritmus: a fogyár forgalmi tényadatból kiszámoljuk az egységárat, majd annak a nettó értékét és abból határozzuk meg az egységnyi árrést egy függvénnyel - az egységnyi árrésből a dobozszámmal felszorolva kapjuk a tételen jelentkezett árrés-forintot, felösszegezve pedig a havi és országos árrés-tömeget:
  - nettósítás: ÁFA levonása (osztás 1.05-tel)
  - egységnyi fogyár számítása
  - árrés-mérték számítása
  - visszaszorzás a darabszámmal
  - felösszegzés

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime, timezone

In [12]:
def margin(price, unit):
    """árrés számoló függyvény, 5 %-os ÁFA figyelembe vételével"""
    netto = price / 1.05
    unitPrice = netto / unit
    unitMargin = 0
    if unitPrice < 637:
        unitMargin = unitPrice * 0.212
    elif unitPrice < 727:
        unitMargin = 136
    elif unitPrice < 1846:
        unitMargin = unitPrice * 0.186
    elif unitPrice < 2086:
        unitMargin = 345
    elif unitPrice < 4201:
        unitMargin = unitPrice * 0.166
    elif unitPrice < 4616:
        unitMargin = 700
    elif unitPrice < 6491:
        unitMargin = unitPrice * 0.152
    else:
        unitMargin = 990
    return unit * unitMargin

In [17]:
db = sqlite3.connect('neak2.sqlite3')

In [18]:
# első adatfeltöltés

years = ['whole2022', 'whole2021', 'whole2020', 'whole2019', 'whole2018', 'whole2017', 'whole2016', 'whole2015', 'whole2014', 'whole2013']
for year in years:
    q = "select idoszak, TTT, fogyar, doboz from "+year+" where venytipus = 'G' and fogyar > 0 and doboz > 0"
    df = pd.read_sql(q, db)
    df['TTTmargin'] = df.apply(lambda df: margin(df['fogyar'], df['doboz']), axis=1)
    margins = df.groupby(['idoszak'], as_index=False)[['fogyar', 'TTTmargin']].agg('sum')
    margins['year'] = margins['idoszak'].str.slice(0,4)
    margins.to_sql(("margins"), db, if_exists = 'append', index=False)

In [22]:
db.close()

# régebbi eljárások

In [13]:
# lekérdezés évenként az árrés számoláshoz: 2022
q = "select idoszak, TTT, fogyar, doboz from whole2022 where venytipus = 'G' and fogyar > 0 and doboz > 0"
df = pd.read_sql(q, db)
df

,idoszak,TTT,fogyar,doboz
0,202201,210751477,67940.0,1.0
1,202201,210712376,19465.6,0.8
2,202201,210712376,1085207.2,44.6
3,202201,210712376,187356.4,7.7
4,202201,210688420,1170666.0,14.0
...,...,...,...,...
690158,202207,210037572,59648.0,8.0
690159,202207,210579443,1900500.0,2172.0
690160,202207,210579443,141750.0,162.0
690161,202207,210579443,110250.0,126.0


In [17]:
# tételes árrés-mező számolása

df['TTTmargin'] = df.apply(lambda df: margin(df['fogyar'], df['doboz']), axis=1)

In [18]:
df

,idoszak,TTT,fogyar,doboz,TTTmargin
0,202201,210751477,67940.0,1.0,990.0
1,202201,210712376,19465.6,0.8,792.0
2,202201,210712376,1085207.2,44.6,44154.0
3,202201,210712376,187356.4,7.7,7623.0
4,202201,210688420,1170666.0,14.0,13860.0
...,...,...,...,...,...
690158,202207,210037572,59648.0,8.0,7920.0
690159,202207,210579443,1900500.0,2172.0,336660.0
690160,202207,210579443,141750.0,162.0,25110.0
690161,202207,210579443,110250.0,126.0,19530.0


In [23]:
margins = df.groupby(['idoszak'])[['fogyar', 'TTTmargin']].agg('sum')

In [24]:
margins

,fogyar,TTTmargin
idoszak,,
202201,4.641184e+10,4.221364e+09
202202,4.586869e+10,4.212125e+09
202203,5.153538e+10,4.698988e+09
202204,4.675286e+10,4.222657e+09
202205,4.957413e+10,4.447066e+09
202206,4.954456e+10,4.415975e+09
202207,4.673400e+10,4.149267e+09


In [25]:
margins.to_sql(("margins"), db, if_exists = 'append')

In [7]:
# lekérdezés évenként az árrés számoláshoz: 2021
q = "select idoszak, TTT, fogyar, doboz from whole2021 where venytipus = 'G' and fogyar > 0 and doboz > 0"
df = pd.read_sql(q, db)
df

,idoszak,TTT,fogyar,doboz
0,202101,210712376,26765.2,1.1
1,202101,210712376,2433.2,0.1
2,202101,210712376,1515883.6,62.3
3,202101,210712376,111927.2,4.6
4,202101,210688420,1254285.0,15.0
...,...,...,...,...
1199757,202112,210579443,128625.0,147.0
1199758,202112,210579443,114625.0,131.0
1199759,202112,210362252,167481.0,3.0
1199760,202112,210362260,998664.0,12.0


In [14]:
df['TTTmargin'] = df.apply(lambda df: margin(df['fogyar'], df['doboz']), axis=1)
margins = df.groupby(['idoszak'], as_index=False)[['fogyar', 'TTTmargin']].agg('sum')
margins['year'] = margins['idoszak'].str.slice(0,4)
margins.to_sql(("margins"), db, if_exists = 'append', index=False)
margins

,idoszak,fogyar,TTTmargin,year
0,202101,4.354498e+10,4.031468e+09,2021
1,202102,4.282885e+10,3.992401e+09,2021
2,202103,4.975911e+10,4.672498e+09,2021
3,202104,4.419874e+10,4.089832e+09,2021
4,202105,4.431823e+10,4.070782e+09,2021
5,202106,4.712836e+10,4.290263e+09,2021
6,202107,4.550667e+10,4.145350e+09,2021
7,202108,4.504021e+10,4.104323e+09,2021
8,202109,4.800083e+10,4.426858e+09,2021
9,202110,4.778746e+10,4.401163e+09,2021


In [13]:
margins['year'] = margins['idoszak'].str.slice(0,4)
margins

,idoszak,fogyar,TTTmargin,year
0,202101,4.354498e+10,4.031468e+09,2021
1,202102,4.282885e+10,3.992401e+09,2021
2,202103,4.975911e+10,4.672498e+09,2021
3,202104,4.419874e+10,4.089832e+09,2021
4,202105,4.431823e+10,4.070782e+09,2021
5,202106,4.712836e+10,4.290263e+09,2021
6,202107,4.550667e+10,4.145350e+09,2021
7,202108,4.504021e+10,4.104323e+09,2021
8,202109,4.800083e+10,4.426858e+09,2021
9,202110,4.778746e+10,4.401163e+09,2021


In [28]:
# lekérdezés évenként az árrés számoláshoz: 2020
q = "select idoszak, TTT, fogyar, doboz from whole2020 where venytipus = 'G' and fogyar > 0 and doboz > 0"
df = pd.read_sql(q, db)
df

,idoszak,TTT,fogyar,doboz
0,202001,210712376,82728.8,3.4
1,202001,210712376,14599.2,0.6
2,202001,210712376,1776236.0,73.0
3,202001,210712376,201955.6,8.3
4,202001,210688420,1170666.0,14.0
...,...,...,...,...
1221401,202012,210579443,139125.0,159.0
1221402,202012,210362252,558270.0,10.0
1221403,202012,210362260,1248330.0,15.0
1221404,202012,210362278,1991070.0,18.0


In [29]:
df['TTTmargin'] = df.apply(lambda df: margin(df['fogyar'], df['doboz']), axis=1)
margins = df.groupby(['idoszak'])[['fogyar', 'TTTmargin']].agg('sum')
margins.to_sql(("margins"), db, if_exists = 'append')
margins

,fogyar,TTTmargin
idoszak,,
202001,4.739648e+10,4.566371e+09
202002,4.435685e+10,4.319860e+09
202003,5.825443e+10,5.821384e+09
202004,4.202585e+10,3.941144e+09
202005,3.994323e+10,3.762448e+09
202006,4.327256e+10,4.067792e+09
202007,4.501677e+10,4.235177e+09
202008,4.191908e+10,3.950535e+09
202009,4.670200e+10,4.404221e+09


In [30]:
# lekérdezés évenként az árrés számoláshoz: 2019
q = "select idoszak, TTT, fogyar, doboz from whole2019 where venytipus = 'G' and fogyar > 0 and doboz > 0"
df = pd.read_sql(q, db)
df

,idoszak,TTT,fogyar,doboz
0,201901,210712376,97328.0,4.0
1,201901,210712376,14599.2,0.6
2,201901,210712376,2019556.0,83.0
3,201901,210712376,287117.6,11.8
4,201901,210688420,1170666.0,14.0
...,...,...,...,...
1239373,201912,210621981,80070.0,3.0
1239374,201912,210362260,1664440.0,20.0
1239375,201912,210362260,416110.0,5.0
1239376,201912,210362278,1880455.0,17.0


In [31]:
df['TTTmargin'] = df.apply(lambda df: margin(df['fogyar'], df['doboz']), axis=1)
margins = df.groupby(['idoszak'])[['fogyar', 'TTTmargin']].agg('sum')
margins.to_sql(("margins"), db, if_exists = 'append')
margins

,fogyar,TTTmargin
idoszak,,
201901,4.494445e+10,4.470895e+09
201902,4.040319e+10,3.987773e+09
201903,4.220426e+10,4.120066e+09
201904,4.362709e+10,4.207311e+09
201905,4.495860e+10,4.353240e+09
201906,4.057499e+10,3.926494e+09
201907,4.367259e+10,4.193923e+09
201908,4.164179e+10,4.030170e+09
201909,4.309787e+10,4.182560e+09


In [34]:
# lekérdezés évenként az árrés számoláshoz: 2018
q = "select idoszak, TTT, fogyar, doboz from whole2018 where venytipus = 'G' and fogyar > 0 and doboz > 0"
df = pd.read_sql(q, db)
df

,idoszak,TTT,fogyar,doboz
0,201801,210712376,97328.0,4.0
1,201801,210712376,14599.2,0.6
2,201801,210712376,2571892.4,105.7
3,201801,210712376,287117.6,11.8
4,201801,210688420,1672380.0,20.0
...,...,...,...,...
1261522,201812,210362252,279135.0,5.0
1261523,201812,210362252,55827.0,1.0
1261524,201812,210362260,3079214.0,37.0
1261525,201812,210362278,2875990.0,26.0


In [35]:
df['TTTmargin'] = df.apply(lambda df: margin(df['fogyar'], df['doboz']), axis=1)
margins = df.groupby(['idoszak'])[['fogyar', 'TTTmargin']].agg('sum')
margins.to_sql(("margins"), db, if_exists = 'append')
margins

,fogyar,TTTmargin
idoszak,,
201801,4.092793e+10,4.226284e+09
201802,3.765165e+10,3.934187e+09
201803,4.097300e+10,4.208904e+09
201804,3.885772e+10,3.931411e+09
201805,4.240387e+10,4.244620e+09
201806,4.029564e+10,4.035929e+09
201807,4.096306e+10,4.037250e+09
201808,4.118031e+10,4.062422e+09
201809,3.931731e+10,3.917581e+09


In [36]:
db.close()